In [1]:
#Data saved in csv/excel
import glob, os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Set two decimal points for display
pd.options.display.float_format = "{:.2f}".format

# 1. Load the data

In [3]:
data=pd.read_csv('C:/Users/tsb7592/Downloads/Basic_Desc_output.csv')

In [4]:
data['PIN']=data['PIN'].str.upper()

In [5]:
data.head()

,PIN,DeviceID,Assessment Name,Inst,ItemID,Response,Score,Position,ResponseTime,DateCreated,InstStarted,InstEnded,Age_month,Age_group
0,TX009C,E2909A06-4CF6-4F3A-A839-A30AE05F2B5B,Assessment 2,Mullen Receptive,Mullen_Rec_Title,1,NaN,1,NaN,2021-12-05 08:30:42,2021-12-05 08:30:41,2021-12-05 08:33:49,18.00,"(15, 18]"
1,TX009C,E2909A06-4CF6-4F3A-A839-A30AE05F2B5B,Assessment 2,Mullen Receptive,Mullen_Rec_Instructions,1,NaN,2,NaN,2021-12-05 08:30:43,2021-12-05 08:30:41,2021-12-05 08:33:49,18.00,"(15, 18]"
2,TX009C,E2909A06-4CF6-4F3A-A839-A30AE05F2B5B,Assessment 2,Mullen Receptive,RL13,2,0.00,3,NaN,2021-12-05 08:32:53,2021-12-05 08:30:41,2021-12-05 08:33:49,18.00,"(15, 18]"
3,TX009C,E2909A06-4CF6-4F3A-A839-A30AE05F2B5B,Assessment 2,Mullen Receptive,RL10,1,1.00,4,NaN,2021-12-05 08:32:59,2021-12-05 08:30:41,2021-12-05 08:33:49,18.00,"(15, 18]"
4,TX009C,E2909A06-4CF6-4F3A-A839-A30AE05F2B5B,Assessment 2,Mullen Receptive,RL15,2,0.00,5,NaN,2021-12-05 08:33:18,2021-12-05 08:30:41,2021-12-05 08:33:49,18.00,"(15, 18]"


In [7]:
data.Inst.unique()

array(['Mullen Receptive', 'Mullen Expressive Prompted',
       'Executive Function', 'Verbal Counting', 'Toy Out of Reach',
       'Social Observation', 'Playing with Bubbles',
       'Looking While Listening', 'Get Up and Go', 'Sit and Stand',
       'Numerical Change Detection', 'Shape Matching Manipulatives',
       'Reach To Eat', 'Spatial Change Detection',
       'Make Yours Look Like Mine', 'Mullen Expressive Observational',
       'Global Self-Regulation', 'Object Counting', 'Give_N',
       'Memory Task Learning', 'Visual Delayed Response',
       'Memory Task Test',
       'NIH Baby Toolbox Picture Vocabulary Test Practice',
       'NIH Baby Toolbox Picture Vocabulary Test', 'Who Has More',
       'Shape Matching', 'Number Recognition + Subitizing',
       'Early Childhood Behavior Questionnaire - Very Short Form',
       'Caregiver Checklist',
       'PROMIS EC Parent-Report SF v1.0 - Social Relationships - Child-Caregiver Interactions 5a',
       'PROMIS EC Parent-Report S

In [8]:
df= data[data.Inst=='Get Up and Go']

In [15]:
df.shape

(3717, 14)

In [71]:
df.PIN=df.PIN.str[:5]

# 2. Create a 2-dimensional array—first dimension is the age-groups, second dimension is every single ItemID. Values within the array are counts for number of subjects that took that item. Use the `Response` field to determine if it was administered or not; if any response is recorded, add that participant to the relevant cell. A person cannot be counted more than once. 

In [16]:
df =df[df.Response.notnull()]

In [11]:
df.groupby('ItemID')['Age_group'].count()

ItemID
GuD-Instct    41
GuD-PBack     41
GuD-Start     41
GuD-Step      41
GuD-VdInst    41
              ..
GuS-Toe       67
GuS-Turn       3
GuS-Up        77
GuS-VidT      82
GuS-Video     82
Name: Age_group, Length: 68, dtype: int64

# 3. Overall only (not by age group), if the `Response` field is an INTEGER, tabulate the values within an ItemID across all individuals. Note that some `Response` fields are strings or floats, in which case that ItemID does not require a table. 

In [25]:
intrp = df[df['Response'].str.isnumeric()]

In [26]:
# one column for each of the ~35 Inst [instrument names]
intrp.groupby(['ItemID'])['Response'].apply(set)

ItemID
GuD-Instct                              {1}
GuD-PBack                               {2}
GuD-Step                    {4, 7, 6, 5, 2}
GuD-VdInst                              {1}
GuF-BackL                            {1, 2}
GuF-BackR                            {1, 2}
GuF-FallL                               {2}
GuF-FallR                            {1, 2}
GuF-FiveL                            {1, 2}
GuF-FiveR                            {1, 2}
GuF-FootR                               {3}
GuF-LInstL                              {1}
GuF-LInstR                              {1}
GuF-LMethL                        {5, 1, 6}
GuF-LMethR                        {5, 6, 3}
GuF-PBackL                              {2}
GuF-PBackR                              {2}
GuF-PinstL                              {1}
GuF-PinstR                              {1}
GuF-PreBkL                              {2}
GuF-PreBkR                              {2}
GuF-PreQ                             {1, 2}
GuF-PrevidTL             

# 4. Calculate the locomotion durations for three prompts using the difference between `Response` value

In [71]:
#Off-Floor-Right duration: Take the `Response` field from ItemID GuF-EndR and subtract the `Response` field from ItemID GuF-StartR 

def duration(df, col1, col2, new):
    End = df[df.ItemID==col1][['PIN','Assessment Name','Response']].drop_duplicates()
    Start =df[df.ItemID==col2][['PIN','Assessment Name','Response']].drop_duplicates()
    a = pd.merge(End, Start, on=['PIN','Assessment Name'], how='left')
    
    a['Response_x'] =pd.to_numeric(a['Response_x'])
    a['Response_y'] =pd.to_numeric(a['Response_y'])
    a[new] = a['Response_x']- a['Response_y']
    
    return a

In [72]:
a= duration(df, 'GuF-EndR','GuF-StartR', 'Off-Floor-Right')
a.head()

,PIN,Assessment Name,Response_x,Response_y,Off-Floor-Right
0,TX009C,Assessment 2,83.87,19.32,64.55
1,TX005C,Assessment 2,126.54,22.42,104.11
2,APPLETONC1,Assessment 1,7.02,3.18,3.84
3,MO008C,Assessment 1,32.16,13.75,18.41
4,MN011C,Assessment 1,54.37,46.61,7.76


In [73]:
b= duration(df, 'GuF-EndL','GuF-StartL', 'Off-Floor-Left')
b.head()

,PIN,Assessment Name,Response_x,Response_y,Off-Floor-Left
0,MO009C,Assessment 1,42.42,34.13,8.28
1,WI003C,Assessment 2,43.90,26.72,17.18
2,MO001C,Assessment 1,24.06,18.93,5.13
3,MO016C,Assessment 1,25.37,11.74,13.63


In [74]:
c= duration(df, 'GuS-End','GuS-Start', 'Up-Step')
c.head()

,PIN,Assessment Name,Response_x,Response_y,Up-Step
0,TX009C,Assessment 2,80.09,18.52,61.57
1,TX005C,Assessment 2,74.93,22.70,52.23
2,APPLETONC1,Assessment 1,9.11,4.21,4.90
3,MN011C,Assessment 1,96.34,89.71,6.64
4,TX015C,Assessment 1,39.81,13.63,26.17


# 5. Summarize the durations overall and by group (number of individuals taking the test, mean duration, sd of durations, minimum, 25th percentile, median, 75th percentile, and maximum durations) 

In [70]:
b.describe()

count    4.00
mean    11.06
std      5.38
min      5.13
25%      7.50
50%     10.96
75%     14.52
max     17.18
Name: Off-Floor-Left, dtype: float64

In [69]:
pd.concat([a,b,c], axis=1).describe()

,Off-Floor-Right,Off-Floor-Left,Up-Step
count,70.00,4.00,66.00
mean,18.53,11.06,15.21
std,27.06,5.38,15.18
min,0.20,5.13,2.88
25%,5.26,7.50,6.15
50%,9.05,10.96,9.49
75%,18.49,14.52,18.56
max,156.71,17.18,89.79
